In [ ]:
!ls

sample_data


# Практическое задание №3

загрузка и сведение данных в один датафрейм

In [ ]:
!wget https://github.com/krashivan/working_files/raw/main/PA2HW2/home_page_table.csv

In [ ]:
!wget https://github.com/krashivan/working_files/raw/main/PA2HW2/payment_confirmation_table.csv

In [ ]:
!wget https://github.com/krashivan/working_files/raw/main/PA2HW2/payment_page_table.csv

In [ ]:
!wget https://github.com/krashivan/working_files/raw/main/PA2HW2/search_page_table.csv

In [ ]:
!wget https://github.com/krashivan/working_files/raw/main/PA2HW2/user_table.csv

In [ ]:
import pandas as pd

user_table = pd.read_csv('user_table.csv')
home_page = pd.read_csv('home_page_table.csv')
search_page = pd.read_csv('search_page_table.csv')
payment_page = pd.read_csv('payment_page_table.csv')
payment_confirmation = pd.read_csv('payment_confirmation_table.csv')

home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation'})

from functools import reduce
data_frames = [user_table, home_page, search_page, payment_page, payment_confirmation]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on='user_id', how='left'), data_frames)

df_merged['home_page'] = df_merged['home_page'].apply(lambda x: 1 if x == 'home_page' else 0)
df_merged['search_page'] = df_merged['search_page'].apply(lambda x: 1 if x == 'search_page' else 0)
df_merged['payment_page'] = df_merged['payment_page'].apply(lambda x: 1 if x == 'payment_page' else 0)
df_merged['payment_confirmation'] = df_merged['payment_confirmation'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)

df_merged.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


## Задание 1
Использовать датасет с текущего урока и построить сегментированную воронку конверсии пользователей интернет-магазина по полу.

In [ ]:
df_convfun = df_merged.groupby('sex')[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().unstack('sex').unstack('sex').reset_index()
df_convfun.columns.name = None
df_convfun = df_convfun.rename(columns = {'index':'Page'})

df_convfun

,Page,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [ ]:
from plotly import graph_objects as go

fig = go.Figure()

fig.add_trace(go.Funnel(
    y = df_convfun['Page'],
    x = df_convfun['Female'],
    name = 'Female',
    constraintext='outside',
    textposition = 'inside',
    textinfo = 'value+percent previous'
    ))

fig.add_trace(go.Funnel(
    y = df_convfun['Page'],
    x = df_convfun['Male'],
    name = 'Male',
    textposition = 'auto',
    textinfo = 'value+percent previous'
   ))

fig.update_layout(
    title = 'Воронка конверсии по полу',
    title_x=0.5,
    autosize=False,
    width=1200,
    height=600)

fig.show()

## Задание №2
Создать 2 новые фичи на основе колонки “date”: месяц и день недели

In [ ]:
import datetime
df_merged['date'] = pd.to_datetime(df_merged['date'])

df_merged['month'] = df_merged['date'].dt.month_name()
df_merged['day_of_week'] = df_merged['date'].dt.day_name()

new_order = [0,1,8,9,2,3,4,5,6,7]
df_merged = df_merged[df_merged.columns[new_order]]

df_merged.head()

,user_id,date,month,day_of_week,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,February,Saturday,Desktop,Female,1,0,0,0
1,756838,2015-01-13,January,Tuesday,Desktop,Male,1,0,0,0
2,568983,2015-04-09,April,Thursday,Desktop,Male,1,1,0,0
3,190794,2015-02-18,February,Wednesday,Desktop,Female,1,1,0,0
4,537909,2015-01-15,January,Thursday,Desktop,Male,1,0,0,0


## Задание №3
Определить самый топовый по продажам месяц и день недели с помощью базовых методов агрегации (sum или count)

In [ ]:
df_top_month = df_merged.groupby('month')['payment_confirmation'].sum().idxmax()
df_top_day_of_week = df_merged.groupby('day_of_week')['payment_confirmation'].sum().idxmax()

print(f'Топовый месяц по продажам - {df_top_month}\nТоповый день недели по продажам - {df_top_day_of_week}')

Топовый месяц по продажам - January
Топовый день недели по продажам - Monday


## Задание №4
Выяснить, в какой день недели лучше покупают женщины, а в какой мужчины с помощью одного из методов построения сводных таблиц (groupby, pivot_table или crosstab)

In [ ]:
df_top_day_female = df_merged.loc[(df_merged['sex'] == 'Female') & (df_merged['payment_confirmation'] == 1), ['day_of_week', 'payment_confirmation']].groupby(by=['day_of_week']).count()
df_top_day_female = df_top_day_female.rename(columns = {"payment_confirmation": "Female"})

df_top_day_male = df_merged.loc[(df_merged['sex'] == 'Male') & (df_merged['payment_confirmation'] == 1), ['day_of_week', 'payment_confirmation']].groupby(by=['day_of_week']).count()
df_top_day_male = df_top_day_male.rename(columns = {"payment_confirmation": "Male"})

df_top_day_both = pd.merge(df_top_day_female, df_top_day_male, on = 'day_of_week', how = 'left')

df_top_day_both['More_purchases'] = df_top_day_both.apply(lambda column: 'Female' if column[0] > column[1] else 'Male', axis=1)

df_top_day_both = df_top_day_both.reset_index()

cats = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_top_day_both['day_of_week'] = pd.Categorical(df_top_day_both['day_of_week'], categories=cats, ordered=True)
df_top_day_both = df_top_day_both.sort_values('day_of_week').set_index('day_of_week')
df_top_day_both


,Female,Male,More_purchases
day_of_week,,,
Monday,43,34,Female
Tuesday,40,29,Female
Wednesday,30,24,Female
Thursday,31,35,Male
Friday,26,28,Male
Saturday,36,34,Female
Sunday,35,27,Female
